# Manipulation Avancée de Données avec PySpark

Après avoir vu les bases, nous allons approfondir la manipulation de données. Ce notebook couvre les opérations essentielles que vous utiliserez dans 90% de vos projets Data Engineering : lecture/écriture, nettoyage, et logique conditionnelle.

## 1. Initialisation et Chargement de Données

Nous commençons par initialiser la session Spark.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when, avg, count

spark = SparkSession.builder \
    .appName("Manipulation Avancée") \
    .master("local[*]") \
    .getOrCreate()

# Création d'un jeu de données complexe pour l'exercice
data = [
    (1, "Martin", "Paris", 3500, "2023-01-01", None),
    (2, "Sophie", "Lyon", 4200, "2023-02-15", "Manager"),
    (3, "Paul", "Paris", 3100, "2023-03-10", None),
    (4, "Julie", "Marseille", 5000, "2022-05-20", "Director"),
    (5, "Antoine", "Lyon", 2800, None, "Junior"),
    (6, None, "Paris", 3200, "2023-06-01", "Junior")
]

schema = ["id", "nom", "ville", "salaire", "date_embauche", "titre"]

df = spark.createDataFrame(data, schema=schema)
df.show()

## 2. Opérations sur les Colonnes

En PySpark, on modifie rarement un DataFrame "en place". On crée de nouvelles transformations.

### Ajouter et Renommer des colonnes
Utilisez `.withColumn()` pour ajouter ou modifier une colonne, et `.withColumnRenamed()` pour renommer.

In [ ]:
# Ajouter une colonne 'salaire_annuel' (salaire * 12)
df_calcul = df.withColumn("salaire_annuel", col("salaire") * 12)

# Ajouter une colonne constante 'pays'
df_calcul = df_calcul.withColumn("pays", lit("France"))

# Renommer 'nom' en 'prenom'
df_calcul = df_calcul.withColumnRenamed("nom", "prenom")

df_calcul.show()

### Logique Conditionnelle (Case When)
L'équivalent de `IF / ELSE` ou `CASE WHEN` en SQL se fait avec `when().otherwise()`.

In [ ]:
# Créer une catégorie de salaire
df_categorie = df.withColumn("categorie_salaire", 
    when(col("salaire") > 4000, "Élevé")
    .when(col("salaire") > 3000, "Moyen")
    .otherwise("Faible")
)

df_categorie.select("nom", "salaire", "categorie_salaire").show()

## 3. Gestion des Valeurs Nulles (Missing Values)

Les données réelles contiennent souvent des `null`. Spark propose `fillna` et `dropna`.

In [ ]:
# Remplacer les Nulls dans 'titre' par 'Inconnu' et dans 'nom' par 'Anonyme'
df_clean = df.na.fill({
    "titre": "Inconnu",
    "nom": "Anonyme"
})

# Supprimer les lignes où 'date_embauche' est null
df_clean = df_clean.na.drop(subset=["date_embauche"])

df_clean.show()

## 4. Utilisation du SQL natif

Une des grandes forces de Spark est sa compatibilité SQL. Vous pouvez transformer n'importe quel DataFrame en "Table Temporaire" et écrire du SQL standard.

In [ ]:
# Création de la vue temporaire
df.createOrReplaceTempView("employes")

# Requête SQL
resultat_sql = spark.sql("""
    SELECT ville, AVG(salaire) as salaire_moyen, COUNT(*) as nb_employes
    FROM employes
    GROUP BY ville
    HAVING AVG(salaire) > 3000
""")

resultat_sql.show()

## 5. Lecture et Écriture (I/O)

En production, on n'utilise pas `createDataFrame`. On lit des fichiers (CSV, Parquet, JSON).

In [ ]:
# Écriture au format Parquet (Format colonnaire compressé, standard Big Data)
# mode("overwrite") permet d'écraser le dossier s'il existe déjà
df.write.mode("overwrite").parquet("data_output/employes.parquet")

# Lecture du fichier Parquet
df_parquet = spark.read.parquet("data_output/employes.parquet")
df_parquet.printSchema()

In [ ]:
spark.stop()